In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gp 
import plotly.express as px
import json
from matplotlib.colors import Normalize

In [3]:
#read shapefile
townshp = gp.read_file(r"asset/鄉鎮市區界線(TWD97經緯度)1120928/TOWN_MOI_1120825.shp"
                         ,encoding='UTF-8')#without encoding is ok! 

# 將shp檔案格式轉為geoJson檔案格式
townshp.to_file('asset/鄉鎮市區界線(TWD97經緯度)1120928/TOWN_MOI_1120825.geojson', driver='GeoJSON', epsg=4326)  # 經緯度由epsg3826(TWD97)轉換為epsg4326(WGS84)

In [ ]:
# 利用json模組，讀入geojson檔案：
# 讀取geojson
with open('asset/鄉鎮市區界線(TWD97經緯度)1120928/TOWN_MOI_1120825.geojson', encoding='utf8') as response:
    mapGeo = json.load(response)

# 觀察geojson資料狀況
print(mapGeo['features'][0]['properties'])
print(mapGeo['features'][1]['properties'])

In [ ]:
# 制震宅資料整理
ALL_data = pd.read_excel("asset/ALL_data.xlsx")
ALL_data.drop("Unnamed: 0",axis=1,inplace=True)
selling_cases = len(ALL_data)
#ALL_data["制震宅"]=ALL_data["制震宅"].map({0:False,1:True,None:False})
#AST_data = ALL_data[ALL_data["制震宅"]]
ALL_data["鄉鎮市區"]=ALL_data["縣市"]+ALL_data["區域"]
ALL_data

In [ ]:
ALL_data["單價"] = pd.to_numeric(ALL_data['單價'], errors='coerce')
ALL_country_Price = ALL_data[ALL_data['單位'] == '萬/坪'].groupby('鄉鎮市區')['單價'].agg('mean').reset_index()
ALL_country_Price["鄉鎮市區"] = ALL_country_Price["鄉鎮市區"].str.replace("台", "臺")
ALL_country_Price["單價"] = round(ALL_country_Price["單價"],1)

#此處依據geojson檔案內記載的鄉鎮市區清單資訊，建立一個Pandas表格，待會要將此表做為主表，把價格資料併進來：
# 依據圖資資料建立鄉鎮市區清單
country_df = pd.DataFrame.from_dict([i['properties'] for i in mapGeo['features']])
country_df["CityTown"] = country_df["COUNTYNAME"]+country_df["TOWNNAME"]
ALL_country_Price_table = pd.merge(country_df, ALL_country_Price, left_on='CityTown', right_on='鄉鎮市區', how='left')
ALL_country_Price_table

In [ ]:
# 繪製地坪單價地圖
fig = px.choropleth_mapbox(ALL_country_Price_table,                                                # 資料表
                           geojson=mapGeo,                                    # 地圖資訊
                           locations='TOWNCODE',                           # df要對應geojson的id名稱
                           featureidkey='properties.TOWNCODE',             # geojson對應df的id名稱
                           color='單價',                                      # 顏色區分對象
                           color_continuous_scale='Viridis',                  # 設定呈現的顏色
                           range_color=(round(np.nanmin(ALL_country_Price_table['單價'])),    # 顏色的值域範圍
                                        round(np.nanmax(ALL_country_Price_table['單價']))),   
                           mapbox_style='carto-positron',                    # mapbox地圖格式
                           zoom=6,                                           # 地圖縮放大小: 數字愈大放大程度愈大
                           center={'lat': 23.5832, 'lon': 120.5825},         # 地圖中心位置: 此處設定台灣地理中心碑經緯度
                           opacity=0.5,                                               # 設定顏色區塊的透明度 數值愈大愈不透明
                           hover_data=['鄉鎮市區', "單價"]  # 設定游標指向資訊
                          )
fig.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
fig.show()